In [ ]:
import os
import pandas as pd
from random import choice
import shutil

In [ ]:
imgs =[]
train_imgs = []
test_imgs = []

trainPath = 'workspace/training/images/train'
testPath = 'workspace/training/images/test'
crsPath = 'workspace/training/images'

train_ratio = 0.9
test_ratio = 0.1

%mkdir workspace/training/images/train
%mkdir workspace/training/images/test

for (dirname, dirs, files) in os.walk(crsPath):
    for filename in files:
        if filename.endswith('.jpg'):
            imgs.append(filename)

totalImgCount = len(imgs)

countForTrain = int(len(imgs)*train_ratio)
countForTest = int(len(imgs) - countForTrain)

for x in range(countForTrain):

    fileJpg = choice(imgs)

    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(trainPath, fileJpg))

    imgs.remove(fileJpg)
    train_imgs.append(fileJpg)

for x in range(countForTest):

    fileJpg = choice(imgs)

    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(testPath, fileJpg))

    imgs.remove(fileJpg)
    test_imgs.append(fileJpg)

print('Total images: ', totalImgCount)
print('Images in train dir:', len(os.listdir(trainPath)))
print('Images in test dir:', len(os.listdir(testPath)))

Total images:  36
Images in train dir: 32
Images in test dir: 4


In [ ]:
annotations = pd.read_csv('workspace/training/annotations/annotations.csv')

annotations[annotations.filename.isin(train_imgs)].to_csv('workspace/training/annotations/train_annotations.csv', index=None)
annotations[annotations.filename.isin(test_imgs)].to_csv('workspace/training/annotations/test_annotations.csv', index=None)

In [ ]:
!python workspace/generate_tfrecord.py --csv_input=workspace/training/annotations/train_annotations.csv  --image_dir=workspace/training/images/train --output_path=workspace/training/annotations/train.record
!python workspace/generate_tfrecord.py --csv_input=workspace/training/annotations/test_annotations.csv  --image_dir=workspace/training/images/test --output_path=workspace/training/annotations/test.record

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!cd /content/workspace/training/

In [ ]:
!python model_main.py --alsologtostderr --model_dir=./ --pipeline_config_path=faster_rcnn_inception_v2_coco.config